In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [7]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [10]:
df_preprocess=pd.get_dummies(df, columns=["home_ownership",
                                          "verification_status",
                                          "pymnt_plan",
                                          "initial_list_status",
                                          "next_pymnt_d",
                                           "application_type",
                             "hardship_flag",
                             "debt_settlement_flag"])
months={'Jan-2019':1,
        'Feb-2019':2,
        'Mar-2019':3}

df_preprocess["issue_month"] = df_preprocess["issue_d"].apply(lambda x: months[x])


df_preprocess=df_preprocess.drop(["issue_d"],axis=1)

df_preprocess

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,issue_month
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,0,1,0,1,1,0,1,1,3
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,1,0,1,1,0,1,1,3
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,1,0,1,1,0,1,1,3
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,0,1,0,1,1,0,1,1,3
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,1,0,1,1,0,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,low_risk,9.60,0.0,0.0,9.0,0.0,...,1,0,1,0,1,1,0,1,1,1
68813,12000.0,0.2727,368.37,63000.0,low_risk,29.07,0.0,0.0,8.0,0.0,...,1,0,1,0,1,1,0,1,1,1
68814,5000.0,0.1992,185.62,52000.0,low_risk,14.86,0.0,0.0,5.0,1.0,...,1,0,1,0,1,1,0,1,1,1
68815,40000.0,0.0646,1225.24,520000.0,low_risk,9.96,0.0,1.0,21.0,0.0,...,1,1,0,0,1,1,0,1,1,1


# Split the Data into Training and Testing

In [11]:
# Create our features
# YOUR CODE HERE
X =df_preprocess.drop(columns='loan_status')

# Create our target
y =pd.DataFrame(df_preprocess['loan_status'])
 # YOUR CODE HERE

In [170]:
X

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,issue_month
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,0,1,0,1,1,0,1,1,3
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,0,1,0,1,1,0,1,1,3
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,0,1,0,1,1,0,1,1,3
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,0,1,0,1,1,0,1,1,3
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,1,0,1,1,0,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,1,0,1,0,1,1,0,1,1,1
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,1,0,1,0,1,1,0,1,1,1
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,1,0,1,0,1,1,0,1,1,1
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,1,1,0,0,1,1,0,1,1,1


In [12]:
y[:5]

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk


In [141]:
# Check the balance of our target values


In [13]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [146]:
# Creating a StandardScaler instance.
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
#X_scaler = scaler.fit(X_train)

# Scaling the data.
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)


# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE

from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators =100,random_state=1)
brfc.fit(X_train,y_train)







AttributeError: can't set attribute

In [130]:

y_pred= clf.predict(X_test)
y_pred

AttributeError: 'BalancedRandomForestClassifier' object has no attribute 'n_classes_'

In [14]:
# Making predictions using the testing data
y_pred= rf_model.predict(X_test)
y_pred

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [15]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

0.6780424242143579

In [16]:
# Display the confusion matrix
# YOUR CODE HERE

# Calculating the confusion matrix

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

cm_df=pd.DataFrame(
            cm,index=["Actual high_risk","Actual Low_risk"],columns=["Predicted High_risk","Predicted Low_risk"])
cm_df



,Predicted High_risk,Predicted Low_risk
Actual high_risk,36,65
Actual Low_risk,6,17098


In [17]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.36      1.00      0.50      0.60      0.33       101
   low_risk       1.00      1.00      0.36      1.00      0.60      0.38     17104

avg / total       1.00      1.00      0.36      1.00      0.60      0.38     17205



In [19]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Confusion Matrix


,Predicted High_risk,Predicted Low_risk
Actual high_risk,36,65
Actual Low_risk,6,17098


Accuracy Score : 0.6780424242143579
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.36      1.00      0.50      0.60      0.33       101
   low_risk       1.00      1.00      0.36      1.00      0.60      0.38     17104

avg / total       1.00      1.00      0.36      1.00      0.60      0.38     17205



In [20]:

feature_importance=sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
for feature in feature_importance:
    print(f'{feature[1]}:     {feature[0]})')

total_rec_int:     0.08181236252401976)
last_pymnt_amnt:     0.06797297792054745)
total_rec_prncp:     0.06472843576607538)
total_pymnt_inv:     0.05564204018030598)
total_pymnt:     0.05302920209186219)
out_prncp:     0.022824766330973384)
installment:     0.021520169689796576)
dti:     0.017641859609622123)
out_prncp_inv:     0.017612609725672943)
max_bal_bc:     0.01674881801247916)
loan_amnt:     0.015555121120855208)
mo_sin_old_rev_tl_op:     0.015441559129596482)
mo_sin_old_il_acct:     0.015236315797573821)
total_bal_ex_mort:     0.015132688675145741)
tot_cur_bal:     0.015091303728938705)
tot_hi_cred_lim:     0.014949716921160611)
issue_month:     0.014757722260999782)
revol_bal:     0.01469746246699065)
total_il_high_credit_limit:     0.014559721686872477)
bc_util:     0.01444653658422538)
il_util:     0.01406433591485877)
bc_open_to_buy:     0.013691441176159226)
annual_inc:     0.013415668096203)
avg_cur_bal:     0.013388980451823348)
total_bc_limit:     0.012676494977140962

### Easy Ensemble AdaBoost Classifier

In [161]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE

from imblearn.ensemble import EasyEnsembleClassifier
# Instantiate
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)

# Fit
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [23]:
eec.fit(X_train, y_train)
y_pred=eec.predict(X_test)

In [25]:

# Calculated the balanced accuracy score
# YOUR CODE HERE
ecc_score = balanced_accuracy_score(y_test, y_pred)
ecc_score

0.9313969750576554

In [26]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

cm_df=pd.DataFrame(
            cm,index=["Actual high_risk","Actual Low_risk"],columns=["Predicted High_risk","Predicted Low_risk"])
cm_df


,Predicted High_risk,Predicted Low_risk
Actual high_risk,93,8
Actual Low_risk,992,16112


In [56]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



In [57]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Confusion Matrix


,Predicted High_risk,Predicted Low_risk
Actual high_risk,93,8
Actual Low_risk,992,16112


Accuracy Score : 0.6780424242143579
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

